# Loading The Data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random
import pickle 
from tqdm import tqdm


DATADIR = "../Dataset/PetImages"
# DATADIR = "Testing"
CATEGORIES = ["Dog", "Cat"]
IMG_SIZE = 70



In [2]:
training_data = []


def create_training_data():
    for category in CATEGORIES:  # do dogs and cats

        path = os.path.join(DATADIR, category)  # create path to dogs and cats
        # get the classification  (0 or a 1). 0=dog 1=cat
        class_num = CATEGORIES.index(category)

        for img in tqdm(os.listdir(path)):  # iterate over each image per dogs and cats
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)  # convert to array
                # resize to normalize data size
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                # add this to our training_data
                training_data.append([new_array, class_num])
            except Exception as e:  # in the interest in keeping the output clean...
                pass



create_training_data()

print(len(training_data))


100%|██████████| 12501/12501 [00:31<00:00, 393.80it/s]

24946


## Save in pickle

In [3]:
import pickle
random.shuffle(training_data)


X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
# print(X[0])


pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

# We can always load it in to our current script, or a totally new one by doing:

pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle", "rb")
y = pickle.load(pickle_in)


# create a model and save it

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time


pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0

dense_layers = [0]
layer_sizes = [64]
conv_layers = [3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            

            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())

            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            tensorboard = TensorBoard(log_dir="logs\{}".format(NAME))

            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'],
                          )

            model.fit(X, y,
                      batch_size=32,
                      epochs=10,
                      validation_split=0.3,
                      callbacks=[tensorboard])

model.save('64x3-CNN.model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 17462 samples, validate on 7484 samples
Epoch 1/10
17462/17462 [==============================] - 54s 3ms/sample - loss: 0.6368 - acc: 0.6260 - val_loss: 0.5622 - val_acc: 0.7147
Epoch 2/10
17462/17462 [==============================] - 78s 4ms/sample - loss: 0.5210 - acc: 0.7481 - val_loss: 0.4866 - val_acc: 0.7746
Epoch 3/10
17462/17462 [==============================] - 70s 4ms/sample - loss: 0.4613 - acc: 0.7823 - val_loss: 0.4637 - val_acc: 0.7779
Epoch 4/10
17462/17462 [==============================] - 75s 4ms/sample - loss: 0.4159 - acc: 0.8071 - val_loss: 0.4466 - val_acc: 0.7867
Epoch 5/10
17462/17462 [==============================] - 83s 5ms/sample - loss: 0.3774 - acc: 0.8281 - val_loss: 0.3960 - val_acc: 0.8193
Epoch 6/10
17462/17462 [==============================] - 78s 4ms/sample - loss: 